<a href="https://colab.research.google.com/github/lciraci/AI_Projects/blob/main/_English_toItalian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fsspec==2023.4.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.4.0 which is incompatible.
huggingface-hub 0.31.4 requires fsspec>=2023.5.0, but you have fsspec 2023.4.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124

In [2]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00


# parallel sentences: English → Italian.

### Download and preview the opus_books English-Italian translation dataset
### This script loads a public parallel corpus (EN-IT) from the Hugging Face Hub.
### After running the cell below, you'll see the available splits (like train/test)
### and the number of examples in each. Use this as a starting point for any translation/NLP project!


In [5]:
from datasets import load_dataset
dataset = load_dataset("opus_books", "en-it")
print(dataset)


Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 32332
    })
})


### Show the first translation pair (EN → IT) in the training set

In [ ]:
print(dataset['train'][0])

{'id': '0', 'translation': {'en': 'Source: Project Gutenberg', 'it': 'Source: www.liberliber.it/Audiobook available here'}}


### Update the first translation pair in the training set

In [6]:
from datasets import load_dataset, Dataset

# 1. Load
train_data = dataset['train']

# 2. Update the first sample
train_data_list = list(train_data)
train_data_list[0] = {
    'translation': {
        'en': "I'm italian.",
        'it': "Sono italiano."
    }
}

updated_train_data = Dataset.from_list(train_data_list)

# 3. Now you can use updated_train_data for training!
print(updated_train_data[0])


{'translation': {'en': "I'm italian.", 'it': 'Sono italiano.'}}


In [7]:
# Select first 1000 samples for quick testing
small_train = dataset['train'].select(range(1000))
print(small_train)
print(dataset['train'].column_names)


#explore more option on dataset


Dataset({
    features: ['id', 'translation'],
    num_rows: 1000
})
['id', 'translation']


In [ ]:
# Export the train split to a CSV file
dataset['train'].to_csv("opus_books_train.csv")
from google.colab import files
files.download("opus_books_train.csv")


Creating CSV from Arrow format:   0%|          | 0/33 [00:00<?, ?ba/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Show a specific sample and update the first translation pair in the training


In [8]:
print(dataset['train'][190])
#Update
train_data_list = list(train_data)
train_data_list[0] = {
    'translation': {
        'en': "Well, who am I?"" he asked",
        'it': "Chi sono? Ha chiesto?"
    }
}
updated_train_data = Dataset.from_list(train_data_list)


{'id': '190', 'translation': {'en': '"Well, who am I?" he asked.', 'it': '— Chi sono? — domandò egli.'}}


### Remove the first 5 samples from the training set


In [9]:
# Remove the first 5 samples (indices 0 to 4)
train_data_list = train_data_list[5:]

# Re-create the Dataset
updated_train_data = Dataset.from_list(train_data_list)

### Prepare Training and Test set

In [10]:
split_dataset = updated_train_data.train_test_split(test_size=0.1, seed=42)
tokenized_train = split_dataset['train']
tokenized_val = split_dataset['test']



# Tokenization

In [11]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-it"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-it', vocab_size=80035, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	80034: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [12]:
print(tokenizer)
print("Vocabulary size:", tokenizer.vocab_size)
print("Special tokens:", tokenizer.special_tokens_map)
print("First 10 tokens:", list(tokenizer.get_vocab().items())[:50])


MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-it', vocab_size=80035, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	80034: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
Vocabulary size: 80035
Special tokens: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}
First 10 tokens: [('</s>', 0), ('<unk>', 1), ('.', 2), (',', 3), ('▁the', 4), ("'", 5), ('▁di', 6), ('▁in', 7), ('▁of', 8), ('▁a', 9), ('▁e', 10), ('▁to', 11), ('s', 12), ('▁and', 13), ('▁-', 14), ('▁la', 15), ('▁', 16), ('▁che', 17), ('▁il', 18), ('?', 19), ('▁per', 20), (

In [13]:
sample_text = "My name is Lucio"
token_ids = tokenizer.encode(sample_text)
print("Token IDs:", token_ids)
print("Decoded back:", tokenizer.decode(token_ids))


Token IDs: [888, 842, 24, 6263, 564, 0]
Decoded back: My name is Lucio</s>


### For translation, you tokenize source (en) and target (it).
### When fine-tuning, you’ll want to create input_ids and labels for each sample.

### Tokenize the updated training data for model training
### This cell converts each translation pair into input IDs and attention masks
### for the model, and also prepares target labels for training.
### The 'attention_mask' ensures the model ignores padding tokens during learning.


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-it")

def preprocess_function(batch):
    inputs = [example["en"] for example in batch["translation"]]
    targets = [example["it"] for example in batch["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 1. Split the dataset into 90% train, 10% validation
split_dataset = updated_train_data.train_test_split(test_size=0.1, seed=42)
train_data = split_dataset['train']
val_data = split_dataset['test']

# 2. Tokenize both splits
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_val = val_data.map(preprocess_function, batched=True)



Map:   0%|          | 0/29094 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3233 [00:00<?, ? examples/s]

In [15]:
print(tokenized_train[0])

{'id': '3001', 'translation': {'en': 'The passions may rage furiously, like true heathens, as they are; and the desires may imagine all sorts of vain things: but judgment shall still have the last word in every argument, and the casting vote in every decision.', 'it': "La mia ragione è ferma e dominante, essa non permetterà ai sentimenti di perdersi nel vuoto; la passione potrà urlare furiosamente, da vera pagana com'è; i desiderii potranno inventare una serie di pretesti vani, ma il giudizio trionferà sempre e dirà l'ultima parola."}, 'input_ids': [48, 7573, 12, 294, 2412, 1031, 14003, 25521, 3, 192, 1949, 5640, 3610, 12, 3, 66, 154, 55, 63, 13, 4, 5767, 12, 294, 8002, 57, 23183, 8, 25278, 1042, 21, 139, 7730, 224, 684, 78, 4, 679, 2603, 7, 749, 12755, 3, 13, 4, 19872, 3404, 7, 749, 1412, 2, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [80, 4

# Choose and Load a Pre-trained Translation Model

---



In [16]:
from transformers import AutoModelForSeq2SeqLM

# Load the pre-trained MarianMT model for EN→IT translation
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-it")


pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

### Define training arguments for fine-tuning the translation model
### This configures the output directory, batch size, number of epochs,
### learning rate, logging, checkpoint saving, and precision settings.


In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    fp16=True,  # Set to False if no GPU
)


### Create a data collator to handle dynamic padding and batching for seq2seq training


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


### Set up and run the Hugging Face Trainer to fine-tune the translation model


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# Make a tiny train dataset
from transformers import Trainer

small_train = tokenized_train.select(range(100))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()


<ipython-input-63-48bc96935385>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[80034]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=39, training_loss=2.3062832172100363, metrics={'train_runtime': 347.9493, 'train_samples_per_second': 0.862, 'train_steps_per_second': 0.112, 'total_flos': 5879247667200.0, 'train_loss': 2.3062832172100363, 'epoch': 3.0})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


<ipython-input-38-9a00a97565aa>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,2.711700
200,2.381500


Step,Training Loss
100,2.711700
200,2.381500


KeyboardInterrupt: 

### Evaluate the Model

In [ ]:
taset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 32332
    })
})

In [ ]:
# Step 3: Evaluate your model on the validation set
results = trainer.evaluate(eval_dataset=tokenized_val)
print(results)


Map:   0%|          | 0/29094 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3233 [00:00<?, ? examples/s]

{'eval_loss': 2.635819911956787, 'eval_runtime': 1095.4198, 'eval_samples_per_second': 2.951, 'eval_steps_per_second': 0.37, 'epoch': 3.0}


### BLEU Score	Quality
##### 0 – 20	Bad
##### 20 – 40	Understandable, but not great
##### 40 – 60	Good, but still improvable
##### 60 – 80	Very good, close to human
##### 80 – 100	Excellent, almost human identical

In [ ]:
from sacrebleu import corpus_bleu

# Prepare your references and predictions (lists of sentences)
references = ["Questa è una frase di esempio.", "Un'altra frase da testare."]
predictions = ["Questa è una frase di esempio.", "Ecco una frase di prova."]

# SacreBLEU expects references as a list of lists
bleu = corpus_bleu(predictions, [references])
print("BLEU score:", bleu.score)


BLEU score: 58.84191416232741


### Translate a Few Sample Sentences Manually

In [ ]:
def translate(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    output_tokens = model.generate(**inputs)
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Try with some sentences
english_examples = [
    "",
    "How are you today?",#####
    "I like learning new things.",
    "Where is the nearest train station?"
]

for en in english_examples:
    it = translate(en, model, tokenizer)
    print(f"EN: {en}\nIT: {it}\n")


EN: buenas tarde
IT: buenas tarde

EN: How are you today?
IT: Come stai oggi?

EN: I like learning new things.
IT: Mi piace imparare cose nuove.

EN: Where is the nearest train station?
IT: Dov'è la stazione più vicina?



# Improve Model

In [ ]:
#adding more authentic sentences and casual
updated_train_data
# New casual, authentic sentences (EN → IT)
new_examples = [
    {'translation': {'en': "What's up?", 'it': "Che si dice?"}},
    {'translation': {'en': "I'm starving, let's grab a bite.", 'it': "Ho una fame da lupi, andiamo a mangiare qualcosa."}},
    {'translation': {'en': "No worries, it's all good.", 'it': "Tranquillo, tutto a posto."}},
    {'translation': {'en': "Give me a sec.", 'it': "Dammi un attimo."}},
    {'translation': {'en': "He's always late, classic!", 'it': "È sempre in ritardo, come al solito!"}},
    {'translation': {'en': "Are you kidding me?", 'it': "Mi prendi in giro?"}},
    {'translation': {'en': "I have no clue.", 'it': "Non ne ho la minima idea."}},
    {'translation': {'en': "It’s not my cup of tea.", 'it': "Non fa per me."}},
    {'translation': {'en': "I’m just messing with you.", 'it': "Ti sto solo prendendo in giro."}}
]
from datasets import Dataset

# Convert your existing Dataset to a list, add new examples, then recreate the Dataset
train_data_list = list(updated_train_data)
train_data_list.extend(new_examples)

# Create a new Dataset object with the extended data
updated_train_data = Dataset.from_list(train_data_list)

# 1. Split the dataset into 90% train, 10% validation
split_dataset = updated_train_data.train_test_split(test_size=0.1, seed=42)
train_data = split_dataset['train']
val_data = split_dataset['test']

# 2. Tokenize both splits
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_val = val_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/29103 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3234 [00:00<?, ? examples/s]

In [ ]:
# Make a tiny train dataset
from transformers import Trainer

small_train = tokenized_train.select(range(200))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()



<ipython-input-78-bea7d1a924d2>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=75, training_loss=2.2307281494140625, metrics={'train_runtime': 715.728, 'train_samples_per_second': 0.838, 'train_steps_per_second': 0.105, 'total_flos': 12421632098304.0, 'train_loss': 2.2307281494140625, 'epoch': 3.0})

In [ ]:
# Get the first 20 English sentences from your validation set
batch = val_data.select(range(1000))
en_sentences = [ex["translation"]["en"] for ex in batch]
it_references = [ex["translation"]["it"] for ex in batch]

# Generate model predictions
predictions = []
for en in en_sentences:
    inputs = tokenizer(en, return_tensors="pt", padding=True)
    output_tokens = model.generate(**inputs)
    it_pred = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    predictions.append(it_pred)

# Now use sacrebleu with your true references and your model's predictions
from sacrebleu import corpus_bleu
bleu = corpus_bleu(predictions, [it_references])
print("BLEU score:", bleu.score)


BLEU score: 12.769053636226785
